<a href="https://colab.research.google.com/github/Radperia/SIGNATE_student_2020/blob/master/sklearn_stacking_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

いつもの時間確認

In [91]:
!cat /proc/uptime | awk '{print $1 /60 /60 /24 "days (" $1 / 60 / 60 "h)"}'

0.0336106days (0.806656h)


色々importとcatboost, light gbmのインストール

In [92]:
import pandas as pd
import numpy as np
from scipy.stats import skew
import xgboost as xgb
from sklearn.model_selection import KFold
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.linear_model import LogisticRegression
from math import sqrt
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split
import gc

In [93]:
!pip install catboost
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension

Enabling notebook extension jupyter-js-widgets/extension...
      - Validating: OK


In [94]:
!git clone https://github.com/microsoft/LightGBM
!cd LightGBM && mkdir build && cd build && cmake .. && make -j"$(nproc)"
!cd LightGBM/python-package && python setup.py install --precompile

fatal: destination path 'LightGBM' already exists and is not an empty directory.
mkdir: cannot create directory ‘build’: File exists
running install
running build
running build_py
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/Grammar.txt
INFO:root:Generating grammar tables from /usr/lib/python3.6/lib2to3/PatternGrammar.txt
running egg_info
writing lightgbm.egg-info/PKG-INFO
writing dependency_links to lightgbm.egg-info/dependency_links.txt
writing requirements to lightgbm.egg-info/requires.txt
writing top-level names to lightgbm.egg-info/top_level.txt
reading manifest template 'MANIFEST.in'
no previously-included directories found matching 'build'
writing manifest file 'lightgbm.egg-info/SOURCES.txt'
running install_lib
INFO:LightGBM:Installing lib_lightgbm from: ['../lib_lightgbm.so']
running install_egg_info
removing '/usr/local/lib/python3.6/dist-packages/lightgbm-3.0.0-py3.6.egg-info' (and everything under it)
Copying lightgbm.egg-info to /usr/local/lib/python

In [95]:
from catboost import CatBoostClassifier
from lightgbm import LGBMClassifier

In [96]:
NFOLD = 5
SEED = 42

データの読み込み

In [97]:
train_df = pd.read_csv('./drive/My Drive/SIGNATE/train.csv').fillna(' ')
test_df  = pd.read_csv('./drive/My Drive/SIGNATE/test.csv').fillna(' ')
sample = pd.read_csv('./drive/My Drive/SIGNATE/submit_sample.csv', header=None)

train_text = train_df['description']
test_text = test_df['description']
all_text = pd.concat([train_text, test_text])

sentences = train_df['description'].tolist() + test_df['description'].tolist()
sentences[::500]  # 500個飛ばしに確認

['Executes and writes portions of testing plans, protocols, and documentation for assigned portion of application; identifies and debugs issues with code and suggests changes or improvements.',
 'Knowledge and experience of complex software design for distributed systems in embedded networking/telecommunications projects.',
 'Analyze client business processes of storing and managing data.',
 'Defining tasks, timeline and required resources to deliver on the mechanical design portion of a project brief, with strong accountability for successful completion of agreed upon deliverables.',
 'Familiar with sensors, transducers, physiologic modeling, and transfer functions',
 'Manage, maintain, refresh and upgrade environmental components (including patching, etc.)',
 'Conduct research on cutting-edge techniques and tools in machine learning/deep learning/artificial intelligence',
 'Assist in planning and development of visualizations and analytic aggregations to best portray discovered insig

In [98]:
from keras.preprocessing.text import Tokenizer

tokenizer = Tokenizer(num_words=1000)  # 出現頻度上位{num_words}だけを用いる
tokenizer.fit_on_texts(sentences)
print(list(tokenizer.word_index)[:10] ) # 学習された辞書（出現頻度順）

['and', 'to', 'the', 'of', 'with', 'data', 'in', 'for', 'a', 'business']


In [99]:
# 学習・検証に分けて，ベクトルを生成。加えて，正解ラベルも作成
train_X, test_X = np.split(tokenizer.texts_to_matrix(sentences, mode='binary'),
                           [len(train_df)], axis=0)

train_y = train_df['jobflag'].values - 1  # maps {1, 2, 3 ,4} -> {0, 1, 2, 3}
train_X.shape, train_y.shape, test_X.shape

((2931, 1000), (2931,), (1743, 1000))

参考にした

https://www.kaggle.com/eliotbarr/stacking-test-sklearn-xgboost-catboost-lightgbm

In [100]:
kf = KFold(n_splits = NFOLD, shuffle=True, random_state=SEED)

In [101]:
ntrain = train_X.shape[0]
ntest = test_X.shape[0]

In [102]:
class SklearnWrapper(object):
  def __init__(self, clf, seed=SEED, params=None):
    params['random_state'] = seed
    self.clf = clf(**params)

  def train(self, train_X, train_y):
    self.clf.fit(train_X, train_y)

  def predict(self, x):
    return self.clf.predict_proba(x)[:, 1]  

In [103]:
class CatboostWrapper(object):
  def __init__(self, clf, seed=SEED, params=None):
    params['random_seed'] = seed
    self.clf = clf(**params)

  def train(self, train_X, train_y):
    self.clf.fit(train_X, train_y)

  def predict(self, x):
    return self.clf.predict_proba(x)[:, 1]

In [104]:
class LightGBMWrapper(object):
  def __init__(self, clf, seed=SEED, params=None):
    params['feature_fraction_seed'] = seed
    params['bagging_seed'] = seed
    self.clf = clf(**params)

  def train(self, train_X, train_y):
    self.clf.fit(train_X, train_y)
  
  def predict(self, x):
    return self.clf.predict_proba(x)[:, 1]

In [105]:
class XgbWrapper(object):
  def __init__(self, seed=SEED, params=None):
    self.param = params
    self.param['seed'] = seed
    self.nrounds = params.pop('nrounds', 250)

  def train(self, train_X, train_y):
    dtrain = xgb.DMatrix(train_X, label=train_y)
    self.gbdt = xgb.train(self.param, dtrain, self.nrounds)

  def predict(self, x):
    return self.gbdt.predict(xgb.DMatrix(x))

In [107]:
et_params = {
    'n_jobs': -1,
    'n_estimators': 200,
    'max_features': 0.5,
    'max_depth': 12,
    'min_samples_leaf':2
}

In [108]:
rf_params = {
    'n_jobs': -1,
    'n_estimators': 200,
    'max_features': 0.2,
    'max_depth': 12,
    'min_samples_leaf': 2
}

In [109]:
xgb_params = {
    'seed': SEED,
    'colsample_bytree': 0.7,
    'silent': 1,
    'subsample': 0.7,
    'learning_rate': 0.075,
    'objective': 'binary:logistic',
    'max_depth': 4,
    'num_parallel_tree': 1,
    'min_child_weight': 1,
    'nrounds': 200
}

In [144]:
catboost_params = {
    'iterations': 200,
    'learning_rate': 0.05,
    'depth': 3,
    'subsample': 0.7,
    'scale_pos_weight': 5,
    'eval_metric': 'f1',
    'allow_writing_files': False
}

In [111]:
lightgbm_params = {
    'n_estimators': 200,
    'learning_rate': 0.1,
    'num_leaves': 123,
    'colsample_bytree': 0.8,
    'subsample': 0.9,
    'max_depth': 15,
    'reg_alpha': 0.1,
    'reg_lambda': 0.1,
    'min_split_gain': 0.01,
    'min_child_weight': 2
}

In [112]:
xg = XgbWrapper(seed=SEED, params=xgb_params)
et = SklearnWrapper(clf=ExtraTreesClassifier, seed=SEED, params=et_params)
rf = SklearnWrapper(clf=RandomForestClassifier, seed=SEED, params=rf_params)
cb = CatboostWrapper(clf=CatBoostClassifier, seed=SEED, params=catboost_params)
lg = LightGBMWrapper(clf=LGBMClassifier, seed=SEED, params=lightgbm_params)

In [168]:
from sklearn.ensemble import StackingClassifier

estimators = {
    ('ExtraTreeClf', ExtraTreesClassifier(et_params)),
    ('RandomForestClf', RandomForestClassifier(rf_params)),
    #('CatBoostClf', CatBoostClassifier(catboost_params)),
    #('LGBMClf', LGBMClassifier(lightgbm_params))
}

stacking_clf = StackingClassifier(
    estimators=estimators, final_estimator=LogisticRegression()
)
'''
st_train_X = train_X
st_train_y = train_y

st_train_X, st_valid_X, st_train_y, st_valid_y = train_test_split(
    train_X, train_y, test_size=0.25
)
'''

'\nst_train_X, st_valid_X, st_train_y, st_valid_y = train_test_split(\n    train_X, train_y, test_size=0.25\n)\n'

In [169]:
stacking_clf.fit(train_X, train_y)

ValueError: ignored

In [113]:
train_X

array([[0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.],
       ...,
       [0., 0., 0., ..., 0., 0., 0.],
       [0., 1., 0., ..., 0., 0., 0.],
       [0., 1., 1., ..., 0., 0., 0.]])

In [115]:
train_X.shape

(2931, 1000)

In [114]:
train_y

array([1, 2, 3, ..., 0, 2, 1])

In [116]:
train_y.shape

(2931,)

In [118]:
import lightgbm as lgb
from sklearn import metrics

calc_f1 = lambda y, p: metrics.f1_score(y, p.argmax(axis=1), average='macro')

def macro_f1(pred: np.array, data: lgb.Dataset):
    y = data.get_label()
    pred = pred.reshape(-1, len(y)).T  # -> (N, num_class)

    f1 = calc_f1(y, pred)
    return 'macro_f1', f1, True  # True means "higher is better"

In [119]:
weight = 1 / pd.DataFrame(train_y).reset_index().groupby(0).count().values
weight = weight[train_y].ravel()
weight /= weight.sum()

print(weight)

dtrain = lgb.Dataset(train_X, train_y, weight=weight)

[0.00071839 0.00018169 0.00042882 ... 0.00040064 0.00018169 0.00071839]


In [120]:
my_params={
    'objective': 'multiclass',
    'metric': 'custom',
    'num_class': 4,
    'num_iterations': 2000,
    'learning_rate': 0.0001,
    'max_depth': -1,
    'num_leaves': 50,
    'max_bin': 50,
    'colsample_bytree': 0.8,
    'subsample': 0.8,
    'nthread': -1,
    'bagging_freq': 1,
    'verbose': -1,
    'seed': 42,
}

In [127]:
cvbooster = lgb.cv(my_params, dtrain, return_cvbooster=True, stratified=True, 
        num_boost_round=9999, verbose_eval=100, early_stopping_rounds=500,
        feval=macro_f1)['cvbooster']

/usr/local/lib/python3.6/dist-packages/lightgbm/engine.py:530: UserWarning: Found `num_iterations` in params. Will use it instead of argument
  warnings.warn("Found `{}` in params. Will use it instead of argument".format(alias))


[100]	cv_agg's macro_f1: 0.512025 + 0.0134261
[200]	cv_agg's macro_f1: 0.516635 + 0.0117601
[300]	cv_agg's macro_f1: 0.518163 + 0.0163102
[400]	cv_agg's macro_f1: 0.518728 + 0.0182575
[500]	cv_agg's macro_f1: 0.517901 + 0.0195855
[600]	cv_agg's macro_f1: 0.51919 + 0.0196368
[700]	cv_agg's macro_f1: 0.51881 + 0.0214398
[800]	cv_agg's macro_f1: 0.518284 + 0.0220858


In [129]:
predict = cvbooster.predict(test_X)

In [131]:
print([len(v) for v in predict])

[1743, 1743, 1743, 1743, 1743]


In [135]:
predict

[array([[0.24407967, 0.24081578, 0.25261743, 0.26248712],
        [0.23549668, 0.23934675, 0.28706143, 0.23809514],
        [0.24267931, 0.23783893, 0.25848475, 0.26099701],
        ...,
        [0.24099854, 0.24797197, 0.25723268, 0.25379681],
        [0.24665664, 0.23891392, 0.24566731, 0.26876213],
        [0.23391854, 0.24103733, 0.28779488, 0.23724925]]),
 array([[0.2433442 , 0.2423102 , 0.25324893, 0.26109667],
        [0.23295122, 0.24009846, 0.28962749, 0.23732283],
        [0.24213407, 0.24011043, 0.26136246, 0.25639305],
        ...,
        [0.2394752 , 0.25095719, 0.25785443, 0.25171318],
        [0.24074614, 0.23704207, 0.25334132, 0.26887048],
        [0.23378229, 0.23945663, 0.28972408, 0.23703699]]),
 array([[0.24827369, 0.23782144, 0.25606574, 0.25783913],
        [0.2358614 , 0.24039835, 0.28320741, 0.24053284],
        [0.2410435 , 0.23803226, 0.26041033, 0.26051392],
        ...,
        [0.24039829, 0.24207947, 0.26035785, 0.25716439],
        [0.24307814, 0.236161